In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Project_Data\Supply_chain_Project/SupplyChain_DataHub.csv')
df.head()

,payment_type,profit_per_order,sales_per_customer,category_id,category_name,customer_city,customer_country,customer_id,customer_segment,customer_state,...,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,shipping_date,shipping_mode,label
0,PAYMENT,-32.924488,278.95000,38,Kids' Golf Clubs,Caguas,Puerto Rico,12446.5625,Corporate,PR,...,Caribbean,Martinique,PENDING_PAYMENT,858,38,GolfBuddy VT3 GPS Watch,129.99,42177.500,Second Class,2
1,DEBIT,107.874500,263.98000,17,Cleats,Caguas,Puerto Rico,7782.0170,Corporate,PR,...,East Africa,Copperbelt,COMPLETE,365,17,Perfect Fitness Perfect Rip Deck,59.99,42502.390,Same Day,1
2,PAYMENT,35.770718,109.65013,17,Cleats,Caguas,Puerto Rico,7378.1113,Consumer,PR,...,West Asia,Ankara,PENDING_PAYMENT,365,17,Perfect Fitness Perfect Rip Deck,59.99,42951.266,Standard Class,0
3,PAYMENT,43.587560,113.09000,18,Men's Footwear,Caguas,Puerto Rico,1448.6765,Consumer,PR,...,Central America,Francisco Morazan,PENDING_PAYMENT,403,18,Nike Men's CJ Elite 2 TD Football Cleat,129.99,42181.900,Second Class,2
4,PAYMENT,49.804802,191.98090,9,Cardio Equipment,Madison,EE. UU.,5123.5254,Corporate,WI,...,Central America,Leon,PENDING_PAYMENT,191,9,Nike Men's Free 5.0+ Running Shoe,99.99,42632.820,Standard Class,1


In [6]:
df.columns    # 41 columns are present

Index(['payment_type', 'profit_per_order', 'sales_per_customer', 'category_id',
       'category_name', 'customer_city', 'customer_country', 'customer_id',
       'customer_segment', 'customer_state', 'customer_zipcode',
       'department_id', 'department_name', 'latitude', 'longitude', 'market',
       'order_city', 'order_country', 'order_customer_id', 'order_date',
       'order_id', 'order_item_cardprod_id', 'order_item_discount',
       'order_item_discount_rate', 'order_item_id', 'order_item_product_price',
       'order_item_profit_ratio', 'order_item_quantity', 'sales',
       'order_item_total_amount', 'order_profit_per_order', 'order_region',
       'order_state', 'order_status', 'product_card_id', 'product_category_id',
       'product_name', 'product_price', 'shipping_date', 'shipping_mode',
       'label'],
      dtype='object')

**Dropping Columns that are derived (Can be regenerated during analysis)**

In [7]:
df.drop(columns = ['profit_per_order','sales_per_customer','order_item_profit_ratio','order_profit_per_order','label'], inplace=True)
df.columns

Index(['payment_type', 'category_id', 'category_name', 'customer_city',
       'customer_country', 'customer_id', 'customer_segment', 'customer_state',
       'customer_zipcode', 'department_id', 'department_name', 'latitude',
       'longitude', 'market', 'order_city', 'order_country',
       'order_customer_id', 'order_date', 'order_id', 'order_item_cardprod_id',
       'order_item_discount', 'order_item_discount_rate', 'order_item_id',
       'order_item_product_price', 'order_item_quantity', 'sales',
       'order_item_total_amount', 'order_region', 'order_state',
       'order_status', 'product_card_id', 'product_category_id',
       'product_name', 'product_price', 'shipping_date', 'shipping_mode'],
      dtype='object')

**Breaking Down The Columns Into Respecive tables**

In [8]:
df = df.drop_duplicates(subset='customer_id',keep='last')
customers = df[['customer_id','customer_city','customer_state','customer_zipcode','customer_country','customer_segment']] # needs cleaning on customer_id (duplicats present)
products = df[['product_card_id','product_name','category_id']] #Need to be collapsed to fit the exact number of products available (118)
categories = df[['category_id','category_name','department_name']] # Needs to be collapsed to fit the exact number of categories available (51)
orders = df[['order_id','customer_id','order_date','shipping_date','order_status','order_city','order_state','order_country','market','order_region','shipping_mode','payment_type']]
order_items = df[['order_id','product_card_id','product_price','order_item_quantity','order_item_discount','order_item_discount_rate']]

In [9]:
products['product_card_id'].nunique()

118

In [10]:
categories['category_id'].nunique()

51

In [11]:
products = products.drop_duplicates()
categories = categories.drop_duplicates()

In [12]:
print(products['product_card_id'].shape)
print(categories['category_id'].shape)

(118,)
(51,)


**Handling Date Columns**

In [13]:
orders[['order_date','shipping_date']].head() #Date columns appear to have non_date values. Will generate new ones

,order_date,shipping_date
0,42174.500,42177.500
1,42498.390,42502.390
2,42949.266,42951.266
3,42175.900,42181.900
5,42870.883,42872.883


In [14]:
len(orders)

148992

In [15]:
orders_temp = orders.copy()

In [16]:
start_date = pd.Timestamp("2024-08-01")
end_date   = pd.Timestamp("2025-08-31")  # inclusive

# Full daily date range
daily_range = pd.date_range(start_date, end_date, inclusive='left', freq='D')

# Map month → weight
month_weights = {
    1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1,
    8: 1, 9: 1, 10: 2, 11: 5, 12: 5
}
day_weights = daily_range.month.map(month_weights).astype(float)

day_weights_np = np.array([month_weights[m] for m in daily_range.month])

probabilities = day_weights_np / day_weights_np.sum()

N = len(orders)

# Sample days with seasonal weighting
sampled_days = np.random.choice(
    daily_range,
    size=N,
    replace=True,
    p=probabilities
)

# random time of day
random_seconds = np.random.randint(0, 24*3600, size=N)
random_timedeltas = pd.to_timedelta(random_seconds, unit='s')

orders.loc[:,"order_date"] = pd.to_datetime(sampled_days) + random_timedeltas



# Shipping Dates Based on Shipping Mode
mode_delay = {
    'First Class':    (1, 3),
    'Second Class':   (3, 5),
    'Standard Class': (5, 8),
    'Same Day':       (0, 1), 
}

def get_shipping_delay(mode):
    mode = str(mode).strip().lower()
    low, high = mode_delay.get(mode, (3, 7))
    return np.random.randint(low, high + 1) 


shipping_modes = orders["shipping_mode"].str.lower()
delays = shipping_modes.map(lambda m: get_shipping_delay(m))

# Apply delays
orders.loc[:,"shipping_date"] = orders["order_date"] + pd.to_timedelta(delays, unit="D")


C:\Users\user\AppData\Local\Temp\ipykernel_20616\642725781.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'DatetimeIndex(['2024-09-11 10:02:12', '2024-08-15 10:34:14',
               '2025-02-21 21:43:43', '2024-12-19 13:43:23',
               '2024-12-17 12:44:09', '2024-11-18 03:37:07',
               '2024-12-15 04:06:12', '2025-04-30 22:45:19',
               '2024-11-18 03:27:58', '2025-03-13 15:40:28',
               ...
               '2025-07-30 03:08:39', '2024-12-01 19:02:17',
               '2025-07-10 00:17:36', '2024-09-22 13:02:20',
               '2024-11-19 12:09:10', '2025-06-21 13:49:23',
               '2024-11-23 20:37:51', '2024-12-10 16:26:57',
               '2024-11-01 09:53:29', '2024-12-13 00:26:17'],
              dtype='datetime64[ns]', length=148992, freq=None)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  orders.loc[:,"order_date"] = p

In [17]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148992 entries, 0 to 155487
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   order_id       148992 non-null  float64       
 1   customer_id    148992 non-null  float64       
 2   order_date     148992 non-null  datetime64[ns]
 3   shipping_date  148992 non-null  datetime64[ns]
 4   order_status   148992 non-null  object        
 5   order_city     148992 non-null  object        
 6   order_state    148992 non-null  object        
 7   order_country  148992 non-null  object        
 8   market         148992 non-null  object        
 9   order_region   148992 non-null  object        
 10  shipping_mode  148992 non-null  object        
 11  payment_type   148992 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 14.8+ MB


In [18]:
print(orders.loc[orders['shipping_date'].isna()].shape)
print(orders.loc[orders['order_date'].isna()].shape)
print(orders.loc[orders['order_date'] > orders['shipping_date']].shape)   #Dates are all okay

(0, 12)
(0, 12)
(0, 12)


**New Dimension Tables For Normalization and Data Intergrity**

In [19]:
unique_segments = df['customer_segment'].unique()
unique_segments

array(['Corporate', 'Consumer', 'Home Office'], dtype=object)

In [20]:
segment_mapping = {segment: i+1 for i, segment in enumerate(unique_segments)}
segment_mapping

{'Corporate': 1, 'Consumer': 2, 'Home Office': 3}

In [21]:
customers.loc[:,'segment_id'] = customers['customer_segment'].map(segment_mapping)
customers

C:\Users\user\AppData\Local\Temp\ipykernel_20616\4089245265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers.loc[:,'segment_id'] = customers['customer_segment'].map(segment_mapping)


,customer_id,customer_city,customer_state,customer_zipcode,customer_country,customer_segment,segment_id
0,12446.5625,Caguas,PR,725.00,Puerto Rico,Corporate,1
1,7782.0170,Caguas,PR,725.00,Puerto Rico,Corporate,1
2,7378.1113,Caguas,PR,725.00,Puerto Rico,Consumer,2
3,1448.6765,Caguas,PR,725.00,Puerto Rico,Consumer,2
5,11010.9795,Caguas,PR,725.00,Puerto Rico,Consumer,2
...,...,...,...,...,...,...,...
155483,10456.0340,Milford,CT,19063.57,EE. UU.,Consumer,2
155484,5690.0977,Saint Louis,MO,60617.28,EE. UU.,Consumer,2
155485,11397.8750,Chicago,IL,60617.35,EE. UU.,Consumer,2
155486,7845.9480,Provo,UT,79393.89,EE. UU.,Consumer,2


In [22]:
customers = customers.drop(columns='customer_segment')
customers

,customer_id,customer_city,customer_state,customer_zipcode,customer_country,segment_id
0,12446.5625,Caguas,PR,725.00,Puerto Rico,1
1,7782.0170,Caguas,PR,725.00,Puerto Rico,1
2,7378.1113,Caguas,PR,725.00,Puerto Rico,2
3,1448.6765,Caguas,PR,725.00,Puerto Rico,2
5,11010.9795,Caguas,PR,725.00,Puerto Rico,2
...,...,...,...,...,...,...
155483,10456.0340,Milford,CT,19063.57,EE. UU.,2
155484,5690.0977,Saint Louis,MO,60617.28,EE. UU.,2
155485,11397.8750,Chicago,IL,60617.35,EE. UU.,2
155486,7845.9480,Provo,UT,79393.89,EE. UU.,2


In [23]:
customer_segments = pd.DataFrame({'segment_id': list(segment_mapping.values()),
                                    'customer_segments': list(segment_mapping.keys())})
customer_segments

,segment_id,customer_segments
0,1,Corporate
1,2,Consumer
2,3,Home Office


In [24]:
orders.head()

,order_id,customer_id,order_date,shipping_date,order_status,order_city,order_state,order_country,market,order_region,shipping_mode,payment_type
0,62377.010,12446.5625,2024-09-11 10:02:12,2024-09-15 10:02:12,PENDING_PAYMENT,Fort-de-France,Martinique,Martinica,LATAM,Caribbean,Second Class,PAYMENT
1,41717.266,7782.0170,2024-08-15 10:34:14,2024-08-21 10:34:14,COMPLETE,Luanshya,Copperbelt,Zambia,Africa,East Africa,Same Day,DEBIT
2,58860.490,7378.1113,2025-02-21 21:43:43,2025-02-27 21:43:43,PENDING_PAYMENT,Ankara,Ankara,Turkey,Pacific Asia,West Asia,Standard Class,PAYMENT
3,32101.930,1448.6765,2024-12-19 13:43:23,2024-12-25 13:43:23,PENDING_PAYMENT,Tegucigalpa,Francisco Morazan,Honduras,LATAM,Central America,Second Class,PAYMENT
5,29988.092,11010.9795,2024-12-17 12:44:09,2024-12-22 12:44:09,PENDING_PAYMENT,Sari,Mazandaran,Iran,Pacific Asia,South Asia,Second Class,PAYMENT


In [25]:
unique_payments = orders['payment_type'].unique()
payment_mapping = {payment: i+1 for i, payment in enumerate(unique_payments)}
orders.loc[:,'payment_id'] = orders['payment_type'].map(payment_mapping)
orders.head()


C:\Users\user\AppData\Local\Temp\ipykernel_20616\515711112.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders.loc[:,'payment_id'] = orders['payment_type'].map(payment_mapping)


,order_id,customer_id,order_date,shipping_date,order_status,order_city,order_state,order_country,market,order_region,shipping_mode,payment_type,payment_id
0,62377.010,12446.5625,2024-09-11 10:02:12,2024-09-15 10:02:12,PENDING_PAYMENT,Fort-de-France,Martinique,Martinica,LATAM,Caribbean,Second Class,PAYMENT,1
1,41717.266,7782.0170,2024-08-15 10:34:14,2024-08-21 10:34:14,COMPLETE,Luanshya,Copperbelt,Zambia,Africa,East Africa,Same Day,DEBIT,2
2,58860.490,7378.1113,2025-02-21 21:43:43,2025-02-27 21:43:43,PENDING_PAYMENT,Ankara,Ankara,Turkey,Pacific Asia,West Asia,Standard Class,PAYMENT,1
3,32101.930,1448.6765,2024-12-19 13:43:23,2024-12-25 13:43:23,PENDING_PAYMENT,Tegucigalpa,Francisco Morazan,Honduras,LATAM,Central America,Second Class,PAYMENT,1
5,29988.092,11010.9795,2024-12-17 12:44:09,2024-12-22 12:44:09,PENDING_PAYMENT,Sari,Mazandaran,Iran,Pacific Asia,South Asia,Second Class,PAYMENT,1


In [26]:
orders = orders.drop(columns='payment_type')
orders.head()

,order_id,customer_id,order_date,shipping_date,order_status,order_city,order_state,order_country,market,order_region,shipping_mode,payment_id
0,62377.010,12446.5625,2024-09-11 10:02:12,2024-09-15 10:02:12,PENDING_PAYMENT,Fort-de-France,Martinique,Martinica,LATAM,Caribbean,Second Class,1
1,41717.266,7782.0170,2024-08-15 10:34:14,2024-08-21 10:34:14,COMPLETE,Luanshya,Copperbelt,Zambia,Africa,East Africa,Same Day,2
2,58860.490,7378.1113,2025-02-21 21:43:43,2025-02-27 21:43:43,PENDING_PAYMENT,Ankara,Ankara,Turkey,Pacific Asia,West Asia,Standard Class,1
3,32101.930,1448.6765,2024-12-19 13:43:23,2024-12-25 13:43:23,PENDING_PAYMENT,Tegucigalpa,Francisco Morazan,Honduras,LATAM,Central America,Second Class,1
5,29988.092,11010.9795,2024-12-17 12:44:09,2024-12-22 12:44:09,PENDING_PAYMENT,Sari,Mazandaran,Iran,Pacific Asia,South Asia,Second Class,1


In [27]:
payment_types = pd.DataFrame({'payment_id': list(payment_mapping.values()),
                                    'payment_type': list(payment_mapping.keys())})
payment_types

,payment_id,payment_type
0,1,PAYMENT
1,2,DEBIT
2,3,TRANSFER
3,4,CASH


In [28]:
unique_shipping = orders['shipping_mode'].unique()
shipping_mapping = {shipping: i+1 for i, shipping in enumerate(unique_shipping)}
orders.loc[:,'shipping_id'] = orders['shipping_mode'].map(shipping_mapping)
orders = orders.drop(columns='shipping_mode')
orders.head()

,order_id,customer_id,order_date,shipping_date,order_status,order_city,order_state,order_country,market,order_region,payment_id,shipping_id
0,62377.010,12446.5625,2024-09-11 10:02:12,2024-09-15 10:02:12,PENDING_PAYMENT,Fort-de-France,Martinique,Martinica,LATAM,Caribbean,1,1
1,41717.266,7782.0170,2024-08-15 10:34:14,2024-08-21 10:34:14,COMPLETE,Luanshya,Copperbelt,Zambia,Africa,East Africa,2,2
2,58860.490,7378.1113,2025-02-21 21:43:43,2025-02-27 21:43:43,PENDING_PAYMENT,Ankara,Ankara,Turkey,Pacific Asia,West Asia,1,3
3,32101.930,1448.6765,2024-12-19 13:43:23,2024-12-25 13:43:23,PENDING_PAYMENT,Tegucigalpa,Francisco Morazan,Honduras,LATAM,Central America,1,1
5,29988.092,11010.9795,2024-12-17 12:44:09,2024-12-22 12:44:09,PENDING_PAYMENT,Sari,Mazandaran,Iran,Pacific Asia,South Asia,1,1


In [29]:
shipping_mode = pd.DataFrame({'shipping_id': list(shipping_mapping.values()),
                                    'shipping_mode': list(shipping_mapping.keys())})
shipping_mode

,shipping_id,shipping_mode
0,1,Second Class
1,2,Same Day
2,3,Standard Class
3,4,First Class


**Cleaning Tables**

In [30]:
customers.sample(10)

,customer_id,customer_city,customer_state,customer_zipcode,customer_country,segment_id
48749,2826.48320,Sun Valley,CA,91762.610,EE. UU.,2
93566,9823.61500,San Ramon,CA,48225.516,EE. UU.,2
20707,1695.85510,Escondido,CA,92332.110,EE. UU.,1
62133,800.49786,Lancaster,SC,92682.730,EE. UU.,2
62900,11208.22900,Humacao,PR,725.000,Puerto Rico,1
32438,4660.43550,Long Beach,CA,85281.805,EE. UU.,3
44255,7986.21800,Mentor,OH,60132.297,EE. UU.,2
2533,5745.53800,Caguas,PR,725.000,Puerto Rico,1
146219,956.10440,Caguas,PR,725.000,Puerto Rico,3
60734,9005.80100,Powder Springs,GA,84014.980,EE. UU.,1


In [31]:
customers['customer_country'].unique()   #Customers are all from USA teritory (USA and Puerto Rico)

array(['Puerto Rico', 'EE. UU.'], dtype=object)

In [32]:
customers.loc[customers['customer_country'] == 'EE. UU.', 'customer_country'] = 'United States'
customers['customer_country'].unique()

array(['Puerto Rico', 'United States'], dtype=object)

In [33]:
customers['customer_state'].unique()  # There appears to be a number representing a state

array(['PR', 'MO', 'FL', 'IL', 'NM', 'NJ', 'TX', 'HI', 'NY', 'NV', 'MD',
       'CA', 'CT', 'AZ', 'CO', 'OH', 'TN', 'MN', 'MI', 'PA', 'MT', 'UT',
       'GA', 'VA', 'OR', 'NC', 'IN', 'KS', 'MA', 'WA', 'AR', 'WI', 'LA',
       'DE', 'SC', 'IA', 'KY', 'WV', 'OK', 'RI', '95758', 'AL', 'ND',
       'ID'], dtype=object)

In [34]:
customers.loc[customers['customer_state'] == '95758']  # Upon investigation the number appears to be the Zip Code for California. This might be a data entry error


,customer_id,customer_city,customer_state,customer_zipcode,customer_country,segment_id
2485,163.73442,CA,95758,95459.750,United States,1
12464,2319.03660,CA,95758,44108.082,United States,2
17985,2607.45340,CA,95758,11754.474,United States,2
33560,6064.47750,CA,95758,96737.340,United States,2
45548,768.21770,CA,95758,19129.330,United States,3
51142,5858.76760,CA,95758,11246.868,United States,2
57845,10106.75400,CA,95758,8840.258,United States,3
72342,11769.65600,CA,95758,43166.316,United States,2
78797,10739.53300,CA,95758,93258.945,United States,2
88441,7632.65300,CA,95758,75012.190,United States,2


In [35]:
customers.loc[customers['customer_state'] == '95758', 'customer_state'] = 'CA'
customers['customer_state'].unique()

array(['PR', 'MO', 'FL', 'IL', 'NM', 'NJ', 'TX', 'HI', 'NY', 'NV', 'MD',
       'CA', 'CT', 'AZ', 'CO', 'OH', 'TN', 'MN', 'MI', 'PA', 'MT', 'UT',
       'GA', 'VA', 'OR', 'NC', 'IN', 'KS', 'MA', 'WA', 'AR', 'WI', 'LA',
       'DE', 'SC', 'IA', 'KY', 'WV', 'OK', 'RI', 'AL', 'ND', 'ID'],
      dtype=object)

In [36]:
mask = (customers['customer_city'] == 'CA') & (customers['customer_state'] == 'CA')
top_ca_cities = ["Los Angeles","San Diego","San Jose","San Francisco","Fresno","Sacramento","Long Beach","Oakland"]
customers.loc[mask, 'customer_city'] = np.random.choice(top_ca_cities, size= mask.sum())
customers.loc[customers['customer_city'] == 'CA']


,customer_id,customer_city,customer_state,customer_zipcode,customer_country,segment_id


In [37]:
prod_temp = products['product_card_id'].value_counts()
prod_temp.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [38]:
cat_temp = categories['category_id'].value_counts()
cat_temp.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])

In [39]:
orders

,order_id,customer_id,order_date,shipping_date,order_status,order_city,order_state,order_country,market,order_region,payment_id,shipping_id
0,62377.010,12446.5625,2024-09-11 10:02:12,2024-09-15 10:02:12,PENDING_PAYMENT,Fort-de-France,Martinique,Martinica,LATAM,Caribbean,1,1
1,41717.266,7782.0170,2024-08-15 10:34:14,2024-08-21 10:34:14,COMPLETE,Luanshya,Copperbelt,Zambia,Africa,East Africa,2,2
2,58860.490,7378.1113,2025-02-21 21:43:43,2025-02-27 21:43:43,PENDING_PAYMENT,Ankara,Ankara,Turkey,Pacific Asia,West Asia,1,3
3,32101.930,1448.6765,2024-12-19 13:43:23,2024-12-25 13:43:23,PENDING_PAYMENT,Tegucigalpa,Francisco Morazan,Honduras,LATAM,Central America,1,1
5,29988.092,11010.9795,2024-12-17 12:44:09,2024-12-22 12:44:09,PENDING_PAYMENT,Sari,Mazandaran,Iran,Pacific Asia,South Asia,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
155483,64888.080,10456.0340,2025-06-21 13:49:23,2025-06-27 13:49:23,PROCESSING,Tlalnepantla,Mexico,Mexico,LATAM,Central America,3,4
155484,59790.457,5690.0977,2024-11-23 20:37:51,2024-11-30 20:37:51,COMPLETE,Pekanbaru,Riau,Indonesia,Pacific Asia,Southeast Asia,2,1
155485,11180.109,11397.8750,2024-12-10 16:26:57,2024-12-16 16:26:57,PENDING,Altamura,Apulia,Italy,Europe,Southern Europe,3,3
155486,64127.227,7845.9480,2024-11-01 09:53:29,2024-11-04 09:53:29,CLOSED,Milan,Lombardy,Italy,Europe,Southern Europe,4,1


In [40]:
order_items

,order_id,product_card_id,product_price,order_item_quantity,order_item_discount,order_item_discount_rate
0,62377.010,858,129.99,3.0,77.994000,0.200000
1,41717.266,365,59.99,5.0,35.994000,0.120000
2,58860.490,365,59.99,2.0,10.798200,0.090000
3,32101.930,403,129.99,1.0,19.498500,0.150000
5,29988.092,1073,199.99,1.0,29.998500,0.150000
...,...,...,...,...,...,...
155483,64888.080,627,39.99,5.0,36.689725,0.183495
155484,59790.457,1004,399.98,1.0,3.999800,0.010000
155485,11180.109,1354,29.99,4.0,11.996000,0.100000
155486,64127.227,957,299.98,1.0,4.518509,0.015063


# Tables Available for Analysis

- customers
- products
- categories
- orders
- order_items
- customer_segments
- payment_type
- shipping_mode

**Copying Files To File Folder**

In [ ]:
customers.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\customers.csv', index=False)
products.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\products.csv', index=False)
categories.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\categories.csv', index=False)
orders.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\orders.csv', index=False)
order_items.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\order_items.csv', index=False)
customer_segments.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\customer_segments.csv', index=False)
payment_types.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\payment_types.csv', index=False)
shipping_mode.to_csv(r'C:\Users\user\Desktop\LAST_FOLDER\Postgres_files\shipping_mode.csv', index=False)